# Desafio técnico - Data Engineer
## Candidato: Fernando Zipperer Trevisan
## Data 03/12/2021

---
#### Introdução
Para a resolução deste teste foram seguidos os seguintes passos:
1. Criação de um banco de dados local postgreSQL;
2. Pré tratamento dos dados;
3. Carga dos dados em tabelas do banco criado;
4. Análise de dados com Python;

## 1) Criando o banco de dados postgreSQL

O primeiro passo é instalar o postgres no Linux usando os seguintes comandos do terminal:<br>
`sudo apt update` <br>
`sudo apt install postgresql postgresql-contrib`<br>
<br>
Em seguida vamos acessar o terminar do psql, usando o user padrão *postgres*, que iremos manter neste exercício, através do comando:<br>
`sudo -u postgres psql`<br>
<br>
No terminal do postgres, vamos criar um database, que aqui terá o nome de **eleflow**<br>
`createdb eleflow;`<br>
<br>
Para verificar se ele foi criado, ainda no terminal do psql podemos usar o comando `\list` e ele deverá aparecer na listagem.<br>
<br>
Com o banco criando, usamos o `\q` para sair do terminal do psql e seguiremos para o próximo passo.

## 2) Pré tratamento dos dados
Os dados foram fornecidos em planilhas do Excel com formato `xlsx`e possuiam uma quantidade pequena de dados, como pode ser visto nas imagens a seguir, e em padrões de tabelas.
![Dados do PIB no agronegócio](dados_pib.png)
<br>
**<center>Base de dados do PIB do Ramo Agrícola e Agronegócio</center>**
<br>
<br>
![Dados de produção agrícola](dados_producao.png)
<br>
**<center>Base de dados de produção agrícola (área e produção) de Maio e Junho de 2019</center>**
<br>
<br>
<br>
Por serem bases pequenas, optou-se por fazer o pré tratamento e normalização dos dados no próprio Excel e ao fim gerar arquivos *csv*:
- [Produção agrícola](producao.csv)
- [PIB do Ramo Agrícola](pib_cepea.csv)

## 3) Carga dos dados no banco
Após a normalização dos dados no item anterior, podemos criar as tabelas com os data types corretos.<br>
Para facilitar usaremos uma IDE, como o DBeaver, para executar os comandos SQL a seguir e gerar as tabelas:<br>
**Criar tabela para os dados de PIB --> pib_cepea.csv**
```
create table pib_cepea
(
	ano integer not null,
	insumos double precision not null,
	agropecuaria double precision not null,
	industria double precision not null,
	servicos double precision not null,
	total double precision not null,
	grupo_primario varchar(30) not null,
	grupo_secundario varchar(30) not null
);
```
**Criar tabela para os dados de Produção Agrícula --> producao.csv**
```
create table producao
(
	estado varchar(255) not null,
	"2019" integer,
	maio integer,
	junho integer,
	tipo varchar(20) not null
);

```

<br>
Em seguida, através da IDE podemos clicar com o botão direito do mouse nas tabelas e importas os csv para as mesmas.

## 4) Analisar os dados com Python

Agora iremos analisar os dados através do Python para responder as perguntas feitas:
1. Qual a porcentagem de participação da região Centro-Oeste na produção nacional no primeiro semestre de 2019?
2. Qual estado possui maior área produtiva? 
3. Qual a média de produção mensal para o estado de São Paulo? 
4. Defina a correlação entre a produção agrícola e o PIB do Ramo Agrícola e Pecuário em 2019. Qual segmento possui maior correlação e qual possui menor correlação? Demonstre num gráfico e explique os coeficientes. 


Primeiros vamos importar as bibliotecas necessárias e criar algumas funções para consultar os dados do banco e retorná-los em um DataFrame:

In [1]:
# Importando as bibliotecas necessárias
import psycopg2 #lib para conectar ao postgres
import pandas as pd #lib para análise de dados

# Criar conexão com o banco de dados
def conn_db_eleflow():
    conn_eleflow = psycopg2.connect(
        host="127.0.0.1",
        database="eleflow",
        port="5432",
        user="postgres",
        password="postgres",
    )
    return conn_eleflow

# Executar uma consultar e retornar resultados em um DataFrame

def get_data_postgres(query, connection):
    conn = conn_db_eleflow()
    df = pd.read_sql_query(query, conn)
    return df

Agora vamos usar estas funções para criar nossos DataFrames com os dados.
Por termos datasets pequenos, vamos fazer consultar no banco trazendo todas as colunas e sem filtros no WHERE e trataremos em seguida via Python.

In [2]:
# Consultas SQL
query_pib = "SELECT * FROM pib_cepea"
query_producao = "SELECT * FROM producao"

# Retornar os dados em DataFrames
conn = conn_db_eleflow()
df_pib = get_data_postgres(query=query_pib, connection=conn)
df_producao = get_data_postgres(query=query_producao, connection=conn)
conn.close()

In [3]:
df_pib.info()
df_pib.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ano               144 non-null    int64  
 1   insumos           144 non-null    float64
 2   agropecuaria      144 non-null    float64
 3   industria         144 non-null    float64
 4   servicos          144 non-null    float64
 5   total             144 non-null    float64
 6   grupo_primario    144 non-null    object 
 7   grupo_secundario  144 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 9.1+ KB


,ano,insumos,agropecuaria,industria,servicos,total,grupo_primario,grupo_secundario
0,1996,31597.66303,200211.7777,514755.8826,646536.9825,1393102.306,renda,agronegocio
1,1997,32825.99820,199911.3017,482894.8427,604714.5593,1320346.702,renda,agronegocio
2,1998,31818.41065,200969.7414,455966.0021,576123.4132,1264877.567,renda,agronegocio
3,1999,34784.06716,206952.5872,453199.2288,573119.7313,1268055.614,renda,agronegocio
4,2000,40682.95486,209175.3905,496957.4707,626206.3398,1373022.156,renda,agronegocio


In [4]:
df_producao.info()
df_producao.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   estado  54 non-null     object 
 1   2019    53 non-null     float64
 2   maio    54 non-null     int64  
 3   junho   54 non-null     int64  
 4   tipo    54 non-null     object 
dtypes: float64(1), int64(2), object(2)
memory usage: 2.2+ KB


,estado,2019,maio,junho,tipo
0,ACRE,44487.0,39352,39361,area
1,ALAGOAS,45842.0,80283,80385,area
2,AMAPÁ,21548.0,22442,23242,area
3,AMAZONAS,22930.0,22930,22930,area
4,BAHIA,3130065.0,3107865,3085865,area


Durante o pré tratamento dos dados e a visualização feita acima vamos assumir que:
- Nos dados de produção agrícola a coluna **2019** é a média mensal do ano de 2019;
- Usarem os dados onde a coluna grupo_primario seja igual à tag "PIB" que representa o PIB em R$ milhões correntes

Foi possível perceber tamém que há um valor nulo na coluna "2019" dos dados de produção agrícola e por se tratar de uma coluna numérica vamos substituí-lo por zero:

In [5]:
df_producao["2019"] = df_producao["2019"].fillna(0)

---
#### Respondendo a primeira pergunta
Qual a porcentagem de participação da região Centro-Oeste na produção nacional no primeiro semestre de 2019?

In [6]:
centro_oeste = ["DISTRITO FEDERAL", "GOIÁS", "MATO GROSSO", "MATO GROSSO DO SUL"]
df_centro_oeste = df_producao[(df_producao["tipo"] == "producao") &
                               df_producao["estado"].isin(centro_oeste)]

In [7]:
df_centro_oeste

,estado,2019,maio,junho,tipo
33,DISTRITO FEDERAL,826860.0,860900,863936,producao
35,GOIÁS,24165328.0,25295407,25049399,producao
37,MATO GROSSO,67533553.0,70280101,70280101,producao
38,MATO GROSSO DO SUL,18991372.0,19587130,19587130,producao


In [8]:
prod_mensal_centro_oeste = df_centro_oeste["2019"].sum()

A produção média da região centro oeste, em toneladas, durante um semestre no ano de 2019 foi:

In [9]:
print(f"{prod_mensal_centro_oeste * 6:,}")

669,102,678.0


---
#### Segunda pergunta
Qual estado possui maior área produtiva?

In [10]:
df_area = df_producao[df_producao["tipo"]=="area"]

In [11]:
max_area = df_area["2019"].max()

In [12]:
df_area[df_area["2019"] == max_area].style.format(
    {
        "maio": "{:,d}",
        "junho": "{:,d}",
        "2019": "{:,.1f}"
    }
)

,estado,2019,maio,junho,tipo
10,MATO GROSSO,"16,273,085.0","16,730,548","16,730,548",area


---
#### Terceira pergunta
Qual a média de produção mensal para o estado de São Paulo?

In [13]:
df_prod_SP = df_producao[(df_producao["tipo"] == "producao") & (df_producao["estado"] == "SÃO PAULO")]

In [14]:
media_mensal_SP = df_prod_SP["2019"].values[0]

In [15]:
print(f"Produção média mensal para SP: {media_mensal_SP:,}")

Produção média mensal para SP: 9,224,840.0


---
#### Quarta pergunta
Defina a correlação entre a produção agrícola e o PIB do Ramo Agrícola e Pecuário em 2019. Qual segmento possui maior correlação e qual possui menor correlação? Demonstre num gráfico e explique os coeficientes.